In [185]:
import xml.etree.ElementTree as ET
import numpy as np

## Sleep Accuracy

In [186]:
import pandas as pd

results_list = []
XTIMER_BACKOFFS=[40, 30, 22]
for i, backoff in enumerate(XTIMER_BACKOFFS, start=1):
    file = f"data/sleep_accuracy/xunit_xtimer_usleep_set_XTIMER_BACKOFF_{backoff}_now.xml"
    root = ET.parse(file).getroot()

    for d in root.find("testcase[@name='Measure Xtimer Set Accuracy Microseconds Template']").findall(".//property"):
        duration = d.get("name").split('-')[-1]
        results = [float(v["diff"]) - (int(duration) / 1000_000) for v in eval(d.get("value"))]    # diff from sleep time
        results_list.append({
            'type': 'xtimer_set',
            'backoff': backoff,
            'duration': duration,
            'results': results,
            'mean': np.mean(results),
            'max': np.amax(results),
            'min': np.amin(results)
        })
    

    for d in root.find("testcase[@name='Measure Xtimer Usleep Accuracy Microseconds Template']").findall(".//property"):
        duration = d.get("name").split('-')[-1]
        results = [float(v["diff"]) - (int(duration) / 1000_000) for v in eval(d.get("value"))]    # diff from sleep time
        results_list.append({
            'type': 'xtimer_usleep',
            'backoff': backoff,
            'duration': duration,
            'results': results,
            'mean': np.mean(results),
            'max': np.amax(results),
            'min': np.amin(results)
        })

sleep_accuracy = pd.DataFrame.from_dict(results_list)


In [187]:
import plotly.graph_objects as go

xtimer_set_traces = []
xtimer_usleep_traces = []
for backoff in sleep_accuracy['backoff'].unique():
    xtimer_set = sleep_accuracy.loc[sleep_accuracy['type'] == 'xtimer_set']
    data = xtimer_set[xtimer_set['backoff'] == backoff]
    xtimer_set_traces.append(go.Scatter(x=data['duration'], y=data['mean'], name=f'xtimer_set / {backoff}'))

    xtimer_set = sleep_accuracy.loc[sleep_accuracy['type'] == 'xtimer_usleep']
    data = xtimer_set[xtimer_set['backoff'] == backoff]
    xtimer_usleep_traces.append(go.Scatter(x=data['duration'], y=data['mean'], name=f'xtimer_usleep / {backoff}'))


sleep_accuracy_fig = go.Figure()
sleep_accuracy_fig.add_traces(xtimer_set_traces)
sleep_accuracy_fig.add_traces(xtimer_usleep_traces)
sleep_accuracy_fig.update_layout(
    title="Sleep Accuracy with multiple XTIMER_BACKOFF",
    xaxis_title="Specified Sleep Duration [us]",
    yaxis_title="Real Sleep Duration [s]",
)

# add range slider
sleep_accuracy_fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True,
        )
    )
)

sleep_accuracy_fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=0.99))
sleep_accuracy_fig.write_html("docs/sleep_accuracy.html")
sleep_accuracy_fig.show()

## Sleep Jitter

In [191]:
import statistics

file = "data/sleep_jitter/xunit-10.xml"
root = ET.parse(file).getroot()

DURATION = 60

sleep_jitter = pd.DataFrame(columns=['time', 'trace', 'timer_count', 'background_timers'])
for testcase in root.findall("testcase[@classname='tests_gpio_overhead.Sleep Jitter']"):
    duration = DURATION
    traces = []
    for d in testcase.findall('.//property'):
        name = d.get('name').split('-')
        # if name[-1] == 'focus':
        #     intervals = int(int(eval(d.get('value'))[0]) / 1000000)
        if 'intervals' in name:
            intervals = list(map(lambda i: int(int(i) / 1000000), eval(d.get('value'))))
        elif 'trace' in name:
            value = eval(d.get('value'))
            traces.append(list(map(float, value)))

    # get mean trace
    traces_full = list(zip(*traces))
    traces = [np.mean(t) for t in traces_full]

    timer_count = [1 for i in range(len(traces))]
    for i in range(len(timer_count)):
        timer_count[i] += sum([not((i + 1) % x) for x in intervals])
    xaxis = range(1, DURATION + 1)

    # print(len(xaxis), len(trace), len(timer_count), len(intervals))
    new = pd.DataFrame.from_dict({
        'time': xaxis[2:], 
        'trace': traces[2:],
        # 'trace_full': traces_full[2:],
        'timer_count': list(map(int, timer_count[2:])), 
        'background_timers': [len(intervals) for i in range(len(traces))][2:],
    })
    sleep_jitter = sleep_jitter.append(new)

# print(sleep_jitter)
# print(np.std(sleep_jitter.loc[sleep_jitter['background_timers'] == 30, 'trace']))

In [192]:
# sleep jitter stats min/max/avg/std

sleep_jitter_stats_fig = px.violin(sleep_jitter, x='background_timers', y='trace',
    # points='all',
    color='background_timers',
    title="Sleep Jitter Statistics",
    labels={
        "trace": "Sleep Duration (Seconds)",
        "background_timers": "No. Background Timers",
    })
sleep_jitter_stats_fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=0.99))
sleep_jitter_stats_fig.write_html('docs/sleep_jitter_stats.html')
sleep_jitter_stats_fig.show()
    

In [193]:
import plotly.express as px

sleep_jitter_fig = px.scatter(sleep_jitter, x='time', y='trace',
                                color='background_timers',
                                size=list(sleep_jitter['timer_count']),
                                # size='timer_count',
                                labels={
                                    "trace": "Sleep Duration (seconds)",
                                    "time": "Time Unit (second)",
                                    "background_timers": "No. of Background Timers"
                                },
                                title="Sleep Jitter")

# sleep_jitter_fig = px.line(sleep_jitter, x='time', y='trace', color='background_timers')
sleep_jitter_fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=0.99))
sleep_jitter_fig.write_html('docs/sleep_jitter.html')
sleep_jitter_fig.show()